In [1]:
import os
import pandas as pd 
import numpy as np 
import scipy as scp
import sklearn

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn import metrics 
from sklearn.metrics import confusion_matrix

import statsmodels.api as sm
import matplotlib.pyplot as plt

#from sklearn.grid_search import GridSearchCV
from sklearn.model_selection import GridSearchCV

from imblearn.over_sampling import SMOTE

# 📂DATA : 2 Classes

In [2]:
fold = 3

In [3]:
path = "/home/kannika/code/Rheology2023/Rheology_Blood/DataBlood_Viscosity_TrainML_3Fold_EMclass.csv"
data_feature = pd.read_csv(path)
print(data_feature.shape)
print("-"*100)
print(f"All Fold : {set(data_feature.fold)}")
## Split Train data Set
feature_train = data_feature[data_feature["fold"]!=fold].reset_index(drop=True)
print(f"Train Set : Fold ==> {set(feature_train.fold)}")
print("Train = ", feature_train.shape)
## Split Valid data Set
feature_test = data_feature[data_feature["fold"]==fold].reset_index(drop=True)
print(f"Valiadtion Set : Fold ==> {set(feature_test.fold)}")
print("Validation = ", feature_test.shape)
## Print DataFrame
feature_train.head()

(33, 11)
----------------------------------------------------------------------------------------------------
All Fold : {1, 2, 3}
Train Set : Fold ==> {1, 2}
Train =  (22, 11)
Valiadtion Set : Fold ==> {3}
Validation =  (11, 11)


,Unnamed: 0,Unnamed: 0.1,Code,classes,subclass,classes_binary,MCV,MCH,Hb,typeBEvsBM,fold
0,0,42,HN44,HN,Splenectomy,1.0,61.7,19.7,7.0,E,1
1,1,17,HN21,HN,No_Splenectomy,1.0,64.8,19.7,6.5,E,1
2,2,8,HN13,HN,Splenectomy,1.0,72.6,23.4,5.8,E,1
3,3,43,HN29,HN,No_Splenectomy,1.0,64.2,19.1,6.6,E,1
4,4,50,HN39,HN,Splenectomy,1.0,76.4,22.5,6.4,E,1


In [4]:
X_train = feature_train[['MCV','MCH','Hb']]
y_train = feature_train["typeBEvsBM"]
print(X_train.shape)
print(y_train.shape)
print(y_train[0])

(22, 3)
(22,)
E


In [5]:
print(len(list(set(y_train))))
print(set(y_train))

2
{'M', 'E'}


In [6]:
print("Before OverSampling, counts of label 'BE': {}".format(sum(y_train=='E')))
print("Before OverSampling, counts of label 'BM': {} \n".format(sum(y_train=='M')))

Before OverSampling, counts of label 'BE': 16
Before OverSampling, counts of label 'BM': 6 



# 💡Dealing with Class Imbalance with SMOTE

In [7]:
X_train_smote, y_train_smote = SMOTE().fit_resample(X_train, y_train.ravel())
# sm = SMOTE(random_state=100)
# X_train_smote, y_train_smote = sm.fit_sample(X_train, y_train.ravel())

print('After OverSampling, the shape of train_X: {}'.format(X_train_smote.shape))
print('After OverSampling, the shape of train_y: {} \n'.format(y_train_smote.shape))

print("After OverSampling, counts of label 'BE': {}".format(sum(y_train_smote=='E')))
print("After OverSampling, counts of label 'BM': {}".format(sum(y_train_smote=='M')))

After OverSampling, the shape of train_X: (32, 3)
After OverSampling, the shape of train_y: (32,) 

After OverSampling, counts of label 'BE': 16
After OverSampling, counts of label 'BM': 16


## 🩸 Parameter:  Logistic Regression

In [8]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report,confusion_matrix
import warnings
warnings.filterwarnings('ignore')

In [9]:
LR = LogisticRegression(random_state = 1)

LRparam_grid = {
    'C' : [0.001, 0.01, 0.1, 1],
    'penalty': ['l1', 'l2'],
    'max_iter': list(range(100,500,100)),
    'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']}

LR_search = GridSearchCV(LR, param_grid=LRparam_grid, refit = True, verbose = 1, cv=10, n_jobs = -1)
# fitting the model for grid search 
best_LR = LR_search.fit(X_train_smote, y_train_smote.ravel())

Fitting 10 folds for each of 160 candidates, totalling 1600 fits


In [10]:
# view the results as a pandas DataFrame
best_LRdf = pd.DataFrame(best_LR.cv_results_)[['rank_test_score', 'mean_test_score', 'std_test_score', 'params']]
best_LRdf = best_LRdf.sort_values(by='rank_test_score', ascending=True)
best_LRdf = best_LRdf.reset_index(drop=True)
best_LRdf.head()

,rank_test_score,mean_test_score,std_test_score,params
0,1,0.708333,0.233482,"{'C': 1, 'max_iter': 200, 'penalty': 'l2', 'so..."
1,1,0.708333,0.233482,"{'C': 1, 'max_iter': 300, 'penalty': 'l2', 'so..."
2,1,0.708333,0.233482,"{'C': 1, 'max_iter': 400, 'penalty': 'l2', 'so..."
3,1,0.708333,0.233482,"{'C': 1, 'max_iter': 400, 'penalty': 'l2', 'so..."
4,1,0.708333,0.233482,"{'C': 1, 'max_iter': 100, 'penalty': 'l2', 'so..."


In [11]:
# examine the first result
print("**examine the first result","\n")

print(best_LR.cv_results_['params'][0])
print(best_LR.cv_results_['mean_test_score'][0])

# print the array of mean scores only
print("\n","**print the array of mean scores only","\n")

grid_mean_scores = best_LR.cv_results_['mean_test_score']
print(grid_mean_scores)

# examine the best model
print("\n","**examine the best model","\n")

print(best_LR.best_score_)
print(best_LR.best_params_)
print(best_LR.best_estimator_)

**examine the first result 

{'C': 0.001, 'max_iter': 100, 'penalty': 'l1', 'solver': 'newton-cg'}
nan

 **print the array of mean scores only 

[       nan        nan 0.5               nan 0.36666667 0.49166667
 0.49166667 0.46666667 0.5        0.46666667        nan        nan
 0.5               nan 0.36666667 0.49166667 0.49166667 0.46666667
 0.53333333 0.46666667        nan        nan 0.5               nan
 0.36666667 0.49166667 0.49166667 0.46666667 0.53333333 0.5
        nan        nan 0.5               nan 0.36666667 0.49166667
 0.49166667 0.46666667 0.53333333 0.5               nan        nan
 0.5               nan 0.43333333 0.55       0.55       0.46666667
 0.46666667 0.46666667        nan        nan 0.5               nan
 0.43333333 0.55       0.55       0.46666667 0.46666667 0.46666667
        nan        nan 0.5               nan 0.43333333 0.55
 0.55       0.46666667 0.46666667 0.46666667        nan        nan
 0.5               nan 0.43333333 0.55       0.55       0.466666

In [12]:
#Print the tured parameters and score
print("Tuned Decision Tree Parameters: {}".format(best_LR.best_params_))
print("Best score is {}".format(best_LR.best_score_))

Tuned Decision Tree Parameters: {'C': 1, 'max_iter': 100, 'penalty': 'l2', 'solver': 'newton-cg'}
Best score is 0.7083333333333334


In [14]:
best_params_ =  best_LR.best_params_
C_ = best_params_['C'] 
max_iter_ = best_params_['max_iter'] 
penalty_ = best_params_['penalty']   
solver_ = best_params_['solver']

> ## 🚀 Fit Model and setting parameters

In [15]:
## **- train Fit Model XGBOOST
Logistic = LogisticRegression(random_state = 1, C=C_, max_iter=max_iter_, penalty=penalty_, solver=solver_)
LogisticModel = Logistic.fit(X_train_smote, y_train_smote.ravel()) # train XGBOOST model
LogisticModel

LogisticRegression(C=1, random_state=1, solver='newton-cg')

In [16]:
### Testing Check Parameter
print(LogisticModel.get_params())

{'C': 1, 'class_weight': None, 'dual': False, 'fit_intercept': True, 'intercept_scaling': 1, 'l1_ratio': None, 'max_iter': 100, 'multi_class': 'auto', 'n_jobs': None, 'penalty': 'l2', 'random_state': 1, 'solver': 'newton-cg', 'tol': 0.0001, 'verbose': 0, 'warm_start': False}


> ## #️⃣ Save model

In [17]:
nameclass = "typeBEvsBM_SmoteClasses"

In [18]:
#Save Model
import os
import pickle
import imageio

save_pathimg = f'/media/tohn/HDD/rheology2023/ML_Model/Blood_Viscosity/{nameclass}/fold{fold}'
##**Mkdir Directory 
os.makedirs(save_pathimg, exist_ok=True)     
filename = f"Logis_Model_{nameclass}_fold{fold}.pkl"
Model2Save =  f"{save_pathimg}/{filename}"
print(f"[INFO]: Done!! Save Model as : {Model2Save}")

with open(Model2Save, 'wb') as file:
    pickle.dump(LogisticModel, file)

[INFO]: Done!! Save Model as : /media/tohn/HDD/rheology2023/ML_Model/Blood_Viscosity/typeBEvsBM_SmoteClasses/fold3/Logis_Model_typeBEvsBM_SmoteClasses_fold3.pkl
